In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from sklearn import datasets
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np

In [2]:
# load train and test dataset
def load_dataset():
    # load dataset
    data_x = np.load('fractal_reshapeRecPlot_data_x_aumentated_normalized.npy')
    data_y = np.load('fractal_reshapeRecPlot_data_y_aumentated_normalized.npy')
    #fractal_reshapeRecPlot_data_x_aumentated_normalized.npy
    #percentage of the dataset which will be on TRAIN - TEST - VALIDATION
    TOTAL_IMAGES = len(data_y)
    print("TOTAL_IMAGES = ", TOTAL_IMAGES)
    
    TRAIN_PERCENTAGE = 0.8
    TEST_PERCENTAGE = 0.1
    VALIDATION_PERCENTAGE = 0.1

    train_size = int(TRAIN_PERCENTAGE * TOTAL_IMAGES)
    test_size = int(TEST_PERCENTAGE * TOTAL_IMAGES)
    validation_size = int(VALIDATION_PERCENTAGE * TOTAL_IMAGES)

    train_x = data_x[:train_size] #taking train images from original data
    train_y = data_y[:train_size]
    data_x = data_x[train_size:] #removing train images from original data
    data_y = data_y[train_size:]

    test_x = data_x[:test_size] #taking test images from original data
    test_y = data_y[:test_size]
    data_x = data_x[test_size:] #removing test images from original data
    data_y = data_y[test_size:]

    validation_x = data_x #all the remainder images we can put on validation
    validation_y = data_y

    del data_x #free some unused memory
    del data_y

    print("train_size = ", len(train_y))
    print("test_size = ", len(test_y))
    print("validation_size = ", len(validation_y))
 # one hot encode target values
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    validation_y = to_categorical(validation_y)
    return train_x, train_y, test_x, test_y, validation_x, validation_y

In [3]:
# scale pixels
def prep_pixels(train, test, validation):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    validation_norm = validation.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    validation_norm = validation_norm / 255.0
    # return normalized images
    return train_norm, test_norm, validation_norm

In [4]:
data_x = np.load('fractal_reshapeRecPlot_data_x_aumentated_normalized.npy')
data_y = to_categorical(np.load('fractal_reshapeRecPlot_data_y_aumentated_normalized.npy'))

In [5]:
trainX, trainY, testX, testY, validationX, validationY = load_dataset()
#trainX, testX, validationX = prep_pixels(trainX, testX, validationX)

TOTAL_IMAGES =  1184
train_size =  947
test_size =  118
validation_size =  119


In [6]:
trainX[0]

array([[[0.00392157, 0.        , 0.        ],
        [0.01568627, 0.01568627, 0.01176471],
        [0.03137255, 0.03529412, 0.02745098],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.00392157, 0.        , 0.00392157],
        [0.01568627, 0.01568627, 0.01176471],
        [0.03137255, 0.03529412, 0.02745098],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.00392157, 0.        , 0.00392157],
        [0.01568627, 0.01568627, 0.01176471],
        [0.03137255, 0.03137255, 0.02745098],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       ...,

       [[0.02745098, 0.02745098, 0.02745098],
        [0.01176471, 0.01568627, 0.01568627],
        [0.00784314, 0

In [7]:
#trainX, testX, validationX = prep_pixels(trainX, testX, validationX)

In [8]:
#clf1 = tf.keras.models.load_model("cifar-vgg-baseline-holdout-100epochs_all_transfer.npy")
clf1 = tf.keras.models.load_model("resnet50_100epochs_fractal_augmented_kfold_1.npy")

In [9]:
print(type(clf1))

<class 'keras.engine.sequential.Sequential'>


In [10]:
clf2 = tf.keras.models.load_model("resnet50_100epochs_fractal_augmented_kfold_2.npy")

In [11]:
clf3 = tf.keras.models.load_model("resnet50_100epochs_fractal_augmented_kfold_3.npy")

In [12]:
clf4 = tf.keras.models.load_model("mobilenet_100epochs_fractal_augmented_kfold_1.npy")

In [13]:
clf5 = tf.keras.models.load_model("mobilenet_100epochs_fractal_augmented_kfold_2.npy")

In [14]:
clf6 = tf.keras.models.load_model("mobilenet_100epochs_fractal_augmented_kfold_3.npy")

In [15]:
class emsembleModel:
    def __init__(self, classifiers, numClasses):
        self.classifiers = classifiers
        self.numClasses = numClasses
        
    def predict_by_voting(self, dataX):
        preds = []
        for clf in self.classifiers:
            preds.append(clf.predict(dataX))
        
        resultPred = []
        M = len(self.classifiers)
        for i in range(len(dataX)):
            counting = [0 for _ in range(self.numClasses)]
            current_best_pred = preds[0][i]
            for j in range(M):
                pred = preds[j][i]
                predictedClass = np.argmax(pred)
                counting[predictedClass]+=1
                if counting[predictedClass] == max(counting):
                    current_best_pred = pred
            resultPred.append(current_best_pred)
        
        return resultPred
    
    def evaluate_by_voting(self, dataX, dataY):
        pred = self.predict_by_voting(dataX)
        ok = 0
        for i in range(len(pred)):
            expectedClass = np.argmax(dataY[i])
            predictedClass = np.argmax(pred[i])
            if predictedClass == expectedClass:
                ok += 1
        return ok/len(dataY)

    def predict_by_softmax(self, dataX):
        preds = []
        for clf in self.classifiers:
            preds.append(clf.predict(dataX))
        
        resultsPred = []
        M = len(self.classifiers)
        for i in range(len(dataX)):
            probs = []
            for j in range(M):
                pred = preds[j][i]
                probs.append(pred)
            compressed_probs = np.array(probs).sum(axis=0)
            compressed_probs = compressed_probs/M
            resultsPred.append(compressed_probs)
        
        return resultsPred
    
    def evaluate_by_softmax(self, dataX, dataY):
        pred = self.predict_by_softmax(dataX)
        ok = 0
        for i in range(len(pred)):
            expectedClass = np.argmax(dataY[i])
            predictedClass = np.argmax(pred[i])
            if predictedClass == expectedClass:
                ok += 1
        return ok/len(dataY)
    

In [16]:
classifiers = [clf1, clf2, clf3, clf4, clf5, clf6]

In [17]:
emsemble = emsembleModel(classifiers, 4)

In [38]:
acc = emsemble.evaluate_by_softmax(data_x, data_y)

37/37 [==============================] - 2s 37ms/step


In [39]:
print('Accuracy Emsemble using Softmax: %.3f' % (acc * 100.0))

Accuracy Emsemble using Softmax: 99.916


In [18]:
acc = emsemble.evaluate_by_voting(data_x, data_y)

37/37 [==============================] - 2s 38ms/step


In [19]:
print('Accuracy Emsemble using Voting: %.3f' % (acc * 100.0))

Accuracy Emsemble using Voting: 99.916
